# Compound shell

## Import packages

In [1]:
import pandas as pd
import numpy as np
from astropy import constants
from astronavigation.deflection import *
from astronavigation.planets import Body, SolarSystem
import numpy.linalg as LA

np.set_printoptions(precision=54)

## Useful constants

In [2]:
pc = constants.pc.to('km').value
AU = constants.au.to('km').value
c = constants.c.to('km/s').value
eps = 1/c

# Create Solar System
ss = SolarSystem()

jupiter = ss.getPlanet('jupiter')
r_jup_arc = 20  # mean jupiter radius in arcseconds

sun = ss.getSun()

## Read and clean data

Read data

In [3]:
# path
path = 'Stars_GareqEvent2017_oneTransit_new2.dat'

# read, use delimiter='\s+' to choose every white space as delimiter
stars_data = pd.read_table(path, delimiter='\s+')

# display
pd.set_option('display.max_columns', None)
stars_data.head(10)

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg]
0,2017.145239,2017.145239,3631075715518049024,4.475043e-01,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.552776,-7.831722
1,2017.145239,2017.145239,3631485528413582336,-5.870432e-01,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.463499,-6.796371
2,2017.145239,2017.145239,3631191099815174272,1.157136e-01,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.480483,-7.499324
3,2017.145239,2017.145239,3631256894418771584,-1.456322e-01,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.331641,-7.236695
4,2017.145239,2017.145239,3631266072764288000,-2.651681e-01,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.383427,-7.117590
5,2017.145239,2017.145239,3631298199119733504,-5.711045e-01,0.076116,0.208823,0.297541,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.620517,-6.813578
6,2017.145239,2017.145239,3631241265033227392,-5.069712e-02,0.074388,0.185999,0.254893,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.593303,-7.333806
7,2017.145239,2017.145239,3631239925003381248,-1.514225e-07,0.100975,-0.000004,0.368185,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.405330,-7.382970
8,2017.145239,2017.145239,3631255107712373376,-2.825577e-01,0.065251,0.157291,0.272657,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.566250,-7.101704
9,2017.145239,2017.145239,3631145740665759616,6.684247e-02,0.093621,-0.075513,0.311841,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.328604,-7.449173


Create queary for source_id and add parallaxes to the data

In [4]:
# not need to evaluate this cell if new data has been already downloaded
with open('query.txt', 'w') as f:
    
    star_id = stars_data['starId'].unique()
    query = 'SELECT *\nFROM gaiaedr3.gaia_source\nWHERE '
    
    for id in star_id:
        query += f'source_id = {id} '
        if id != star_id[-1]:
            query += 'OR '
    f.write(query)

In [5]:
# parallax and parallax erro are in mas
new_data = pd.read_csv('1654093971685O-result.csv')
new_data = new_data[['source_id', 'parallax', 'parallax_error']]
new_data = new_data.set_index('source_id')
new_data

,parallax,parallax_error
source_id,,
3631075715518049024,3.409062,0.032620
3631089188831339264,5.459490,0.035637
3631095407943986048,0.243898,0.018328
3631096438736162944,0.535781,0.026916
3631145740665759616,1.217448,0.033738
3631186628753669888,0.400931,0.014865
3631191099815174272,1.352386,0.018157
3631239925003381248,0.472181,0.016357
3631241265033227392,0.758436,0.020345


In [6]:
# add parallax
parallax = []
parallax_err = []
for id in stars_data['starId']:
    p = new_data['parallax']
    err = new_data['parallax_error']
    parallax.append(p.loc[id])
    parallax_err.append(err.loc[id])
stars_data['parallax[mas]'] = parallax
stars_data['parallax_err[mas]'] = parallax_err
stars_data

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg],parallax[mas],parallax_err[mas]
0,2017.145239,2017.145239,3631075715518049024,0.447504,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.552776,-7.831722,3.409062,0.032620
1,2017.145239,2017.145239,3631485528413582336,-0.587043,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.463499,-6.796371,10.354598,0.184979
2,2017.145239,2017.145239,3631191099815174272,0.115714,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.480483,-7.499324,1.352386,0.018157
3,2017.145239,2017.145239,3631256894418771584,-0.145632,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.331641,-7.236695,6.970485,0.017854
4,2017.145239,2017.145239,3631266072764288000,-0.265168,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.383427,-7.117590,1.330410,0.015664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,2017.145239,2017.145240,3631095407943986048,0.228960,0.071818,0.058662,0.318960,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.462561,-7.612432,0.243898,0.018328
138,2017.145239,2017.145240,3631244559272698112,-0.088713,0.049222,-0.037527,0.220813,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.368255,-7.293935,0.488491,0.018010
139,2017.145239,2017.145240,3631244937229820800,-0.079046,0.054070,0.050917,0.226070,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.457351,-7.304351,2.545210,0.030274
140,2017.145239,2017.145240,3631089188831339264,0.347718,0.055859,0.045807,0.233511,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.448593,-7.731080,5.459490,0.035637


Evaluate impact parameter

In [7]:
# star and jupiter coordinates
star = np.array([stars_data['etaS[deg]'], stars_data['zetaS[deg]']])
jup = np.array([stars_data['etaJup[deg]'], stars_data['zetaJup[deg]']])

# impact parameter in units of jupiter radius
stars_data['b_J'] = np.linalg.norm(star - jup, axis=0) * 3600 / r_jup_arc 

Evaluate proper motion

In [8]:
# proper motion is in mas/yr
pm_df = pd.read_csv('1654093971685O-result.csv')
pm_df = pm_df[['source_id', 'pmra', 'pmdec']]
pm_df = pm_df.set_index('source_id')
# add proper motion
pm_ra = []
pm_dec = []
for id in stars_data['starId']:
    ra_dot = pm_df['pmra']
    dec_dot = pm_df['pmdec']
    pm_ra.append(ra_dot.loc[id])
    pm_dec.append(dec_dot.loc[id])
stars_data['pmRA[mas/yr]'] = pm_ra
stars_data['pmDEC[mas/yr]'] = pm_dec
stars_data.head()

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg],parallax[mas],parallax_err[mas],b_J,pmRA[mas/yr],pmDEC[mas/yr]
0,2017.145239,2017.145239,3631075715518049024,0.447504,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.552776,-7.831722,3.409062,0.032620,81.587075,-57.337299,-22.363290
1,2017.145239,2017.145239,3631485528413582336,-0.587043,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.463499,-6.796371,10.354598,0.184979,108.155070,-99.783487,9.701552
2,2017.145239,2017.145239,3631191099815174272,0.115714,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.480483,-7.499324,1.352386,0.018157,20.941453,-9.999470,4.156140
3,2017.145239,2017.145239,3631256894418771584,-0.145632,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.331641,-7.236695,6.970485,0.017854,33.301944,-76.473223,-7.650200
4,2017.145239,2017.145239,3631266072764288000,-0.265168,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.383427,-7.117590,1.330410,0.015664,50.712957,-5.287082,-0.106522


In [9]:
# shift alpha and delta with proper motion
conv = 1e-3/3600  # from mas to deg

stars_data['starRA[deg]_new'] = stars_data['starRA[deg]'] + stars_data['pmRA[mas/yr]']*(stars_data['tAFx']-2016.0)*conv
stars_data['starDec[deg]_new'] = stars_data['starDec[deg]'] + stars_data['pmDEC[mas/yr]']*(stars_data['tAFx']-2016.0)*conv

Evaluate parallax displacement

In [10]:
# need to transform mas to m^-1
conv = 1e-6/pc  # 1e-3/(pc*1e3)
# useful quantities
alpha = np.deg2rad(stars_data['starRA[deg]_new'])
delta = np.deg2rad(stars_data['starDec[deg]_new'])
X = stars_data['gaiaPosX[m]']
Y = stars_data['gaiaPosY[m]']
Z = stars_data['gaiaPosZ[m]']
omega = stars_data['parallax[mas]']

d_alpha = omega*(1/np.cos(delta))*(X*np.sin(alpha) - Y*np.cos(alpha)) * conv
d_delta = omega*(X*np.cos(alpha)*np.sin(delta) +Y*np.sin(alpha)*np.sin(delta) - Z*np.cos(delta)) * conv

star_gaia_dir = np.array([np.cos(alpha + d_alpha)*np.cos(delta + d_delta), np.sin(alpha + d_alpha)*np.cos(delta + d_delta), np.sin(delta + d_delta)])
star_gaia_dir = star_gaia_dir

In [11]:
# add d_alpha, d_delta
stars_data['dRA[deg]'] = np.rad2deg(d_alpha)
stars_data['dDEC[deg]'] = np.rad2deg(d_delta)

# add star_gaia_dir
stars_data['starGCdirX'] = star_gaia_dir[0]
stars_data['starGCdirY'] = star_gaia_dir[1]
stars_data['starGCdirZ'] = star_gaia_dir[2]

Evaluate theta

In [12]:
# useful values
eta_s = np.deg2rad(stars_data['etaS[deg]'])
zeta_s = np.deg2rad(stars_data['zetaS[deg]'])
eta_J = np.deg2rad(stars_data['etaJup[deg]'])
zeta_J = np.deg2rad(stars_data['zetaJup[deg]'])

# gnomonic coordinates
# cos_psi = np.sin(zeta_J)*np.sin(zeta_s) + np.cos(zeta_J)*np.cos(zeta_s)*np.cos(eta_J - eta_s)
# the previous factor can be neglected because we take the fraction
x_g = np.cos(zeta_J)*np.sin(eta_J - eta_s)
y_g = np.sin(zeta_J)*np.cos(zeta_s) - np.cos(zeta_J)*np.sin(zeta_s)*np.cos(eta_J - eta_s)
theta = np.arctan2(y_g,x_g)
stars_data['theta'] = theta

In [13]:
theta

0     -2.853116
1     -0.053039
2     -2.653213
3      0.540839
4      0.159972
         ...   
137   -2.968396
138    0.521361
139   -0.315617
140   -3.067247
141   -2.705232
Length: 142, dtype: float64

Display data

In [14]:
stars_data.head(10)

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg],parallax[mas],parallax_err[mas],b_J,pmRA[mas/yr],pmDEC[mas/yr],starRA[deg]_new,starDec[deg]_new,dRA[deg],dDEC[deg],starGCdirX,starGCdirY,starGCdirZ,theta
0,2017.145239,2017.145239,3631075715518049024,4.475043e-01,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.552776,-7.831722,3.409062,0.032620,81.587075,-57.337299,-22.363290,201.552758,-7.831729,6.739292e-07,-2.531477e-07,-0.921404,-0.363931,-0.136264,-2.853116
1,2017.145239,2017.145239,3631485528413582336,-5.870432e-01,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.463499,-6.796371,10.354598,0.184979,108.155070,-99.783487,9.701552,201.463467,-6.796368,2.039145e-06,-7.352052e-07,-0.924111,-0.363337,-0.118341,-0.053039
2,2017.145239,2017.145239,3631191099815174272,1.157136e-01,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.480483,-7.499324,1.352386,0.018157,20.941453,-9.999470,4.156140,201.480480,-7.499322,2.668142e-07,-9.904283e-08,-0.922583,-0.363052,-0.130514,-2.653213
3,2017.145239,2017.145239,3631256894418771584,-1.456322e-01,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.331641,-7.236695,6.970485,0.017854,33.301944,-76.473223,-7.650200,201.331617,-7.236697,1.370928e-06,-5.051098e-07,-0.924071,-0.360868,-0.125969,0.540839
4,2017.145239,2017.145239,3631266072764288000,-2.651681e-01,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.383427,-7.117590,1.330410,0.015664,50.712957,-5.287082,-0.106522,201.383425,-7.117590,2.618223e-07,-9.587169e-08,-0.923986,-0.361798,-0.123906,0.159972
5,2017.145239,2017.145239,3631298199119733504,-5.711045e-01,0.076116,0.208823,0.297541,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.620517,-6.813578,1.593033,0.015341,110.441922,-9.118550,-20.170386,201.620514,-6.813584,3.145644e-07,-1.130959e-07,-0.923079,-0.365855,-0.118639,-0.311278
6,2017.145239,2017.145239,3631241265033227392,-5.069712e-02,0.074388,0.185999,0.254893,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.593303,-7.333806,0.758436,0.020345,31.851915,4.513326,-8.583775,201.593304,-7.333809,1.498627e-07,-5.510800e-08,-0.922213,-0.365005,-0.127650,-1.202731
7,2017.145239,2017.145239,3631239925003381248,-1.514225e-07,0.100975,-0.000004,0.368185,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.405330,-7.382970,0.472181,0.016357,4.427644,-3.083617,-3.564298,201.405329,-7.382971,9.301377e-08,-3.442134e-08,-0.923303,-0.361938,-0.128501,1.015242
8,2017.145239,2017.145239,3631255107712373376,-2.825577e-01,0.065251,0.157291,0.272657,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.566250,-7.101704,3.914919,0.023334,58.587860,22.925695,-17.254796,201.5662

### Find target stars

In [15]:
# threshold
thr = 5

# create threshold column
stars_data['thr'] = stars_data['b_J'] <= thr

# print number of targets
stars_data['thr'].value_counts()

False    133
True       9
Name: thr, dtype: int64

### Create target and reference dataframe

In [16]:
def ref_targ_split(df):
    
    filt = df['thr']

    target_df = df.loc[filt]
    reference_df = df.loc[(~filt)]
    
    return target_df, reference_df

## Evaluate deflection

In [17]:
# define epochs
epochs = stars_data['tAFx'].unique()

In [18]:
# differences
shell=[]
shell_q=[]
dt_new = []

# loop over epochs
for epoch in epochs:
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    bj_target = target_df['b_J'].to_numpy()
    bj_reference = reference_df['b_J'].to_numpy()
    
    
    shell_epoch = np.zeros((len(target_df), len(reference_df)))
    shell_q_epoch = np.zeros((len(target_df), len(reference_df)))
    
    
    # create body
    #x_b = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T / 1000# in km
    x_b = cartesian(stars_epoch['etaJup[deg]'], stars_epoch['zetaJup[deg]']).T * 5*AU
    
    spin = cartesian(stars_epoch['etaSpin[deg]'].iloc[0], stars_epoch['zetaSpin[deg]'].iloc[0]) 
    # spin = np.array([0, 0, 1])
    
    body = Body(mass=jupiter.mass,
                pos=x_b[0],
                radius=jupiter.radius,
                J2=jupiter.J2,
                s=spin) 
    
    # sun
    x_sun = -np.array([stars_epoch['gaiaPosX[m]'], stars_epoch['gaiaPosY[m]'], stars_epoch['gaiaPosZ[m]']]).T/1000 # in km
    
    sun = Body(mass=sun.mass,
               pos=x_sun[0],
               radius=sun.radius)
    
    ##################
    #
    # print
    #
    ##################
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # loop over target
    #x_targ = np.array([target_df['starGCdirX'], target_df['starGCdirY'], target_df['starGCdirZ']]).T
    x_targ = cartesian(target_df['etaS[deg]'], target_df['zetaS[deg]']).T
    for i in range(len(target_df)):
        
        # direction
        l0 = x_targ[i]
        
        # triplet
        t, n, m = on_triplet(l0, body.pos)
        
        # deflection
        defl_tar = deflection_mod3(l0, body.pos, eps, body.mass)
        dp_eta_tar = defl_tar * np.cos(theta_target[i])
        
        # deflection quadrupole
        dp1_tar, dp2_tar = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
        dp_q_eta_tar = dp1_tar * np.cos(theta_target[i]) + dp2_tar * np.sin(theta_target[i])
        
        # sun correction
        dl_sun = deflection_mod3(l0, sun.pos, eps, sun.mass)
        
        t_targ_new = t+(defl_tar + dp1_tar)*n + dp2_tar*m
        t_targ_new = t_targ_new/LA.norm(t_targ_new)
        
        
        ##################
        #
        # print
        #
        ##################
        print(f'\nb_J: {bj_target[i]}')
        print(f'\ndefl_tar: {np.rad2deg(defl_tar)*3600*1e6} muas')
        print(f'dp1_tar: {np.rad2deg(dp1_tar)*3600*1e6} muas')
        print(f'dp2_tar: {np.rad2deg(dp2_tar)*3600*1e6} muas')
        print(f'dp_eta_tar: {np.rad2deg(dp_eta_tar)*3600*1e6} muas')
        print(f'dp_q_eta_tar: {np.rad2deg(dp_q_eta_tar)*3600*1e6} muas\n')
        # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
        
        
        # loop over reference
        #x_ref = np.array([reference_df['starGCdirX'], reference_df['starGCdirY'], reference_df['starGCdirZ']]).T
        x_ref = cartesian(reference_df['etaS[deg]'], reference_df['zetaS[deg]']).T
        for j in range(len(reference_df)):
            
            # direction
            l0 = x_ref[j]
            
            # triplet
            t, n, m = on_triplet(l0, body.pos)
        
            # deflection
            defl_ref = deflection_mod3(l0, body.pos, eps, body.mass)
            dp_eta_ref = defl_ref * np.cos(theta_reference[i])
            
            diff = dp_eta_tar - dp_eta_ref
            
            # deflection quadrupole
            dp1_ref, dp2_ref = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
            dp_q_eta_ref = dp1_ref * np.cos(theta_reference[j]) + dp2_ref * np.sin(theta_reference[j])
            
            diff_q = dp_q_eta_tar - dp_q_eta_ref
            
            shell_epoch[i, j] = np.rad2deg(diff)*3600*1e6
            shell_q_epoch[i, j] = np.rad2deg(diff_q)*3600*1e6
            
            # sun correction
            dl_sun = deflection_mod3(l0, sun.pos, eps, sun.mass)
            
            t_ref_new = t+(defl_ref + dp1_ref)*n + dp2_ref*m
            t_ref_new = t_ref_new/LA.norm(t_ref_new)
            
            dt_new.append(np.rad2deg(np.arccos(np.dot(t_targ_new, t_ref_new)))*3600*1e6)
            
            ##################
            #
            # print
            #
            ##################
            print(f'\nb_J: {bj_reference[j]}')
            print(f'defl_ref: {np.rad2deg(defl_ref)*3600*1e6} muas')
            print(f'dp1_ref: {np.rad2deg(dp1_ref)*3600*1e6} muas')
            print(f'dp2_ref: {np.rad2deg(dp2_ref)*3600*1e6} muas')
            print(f'dp_eta_ref: {np.rad2deg(dp_eta_ref)*3600*1e6} muas')
            print(f'dp_q_eta_ref: {np.rad2deg(dp_q_eta_ref)*3600*1e6} muas\n')
            # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
            
    
    shell.append(shell_epoch)
    shell_q.append(shell_q_epoch)


---------- epoch: 2017.1452385988 ----------


b_J: 4.427643850555587

defl_tar: 3622.356028983728 muas
dp1_tar: -0.6180035333819498 muas
dp2_tar: -0.17597130634532407 muas
dp_eta_tar: 1910.4829760383607 muas
dp_q_eta_tar: -0.47545065748617776 muas


b_J: 81.58707469920164
defl_ref: 196.5806279085353 muas
dp1_ref: -3.71520299460199e-05 muas
dp2_ref: 9.311492554973134e-05 muas
dp_eta_ref: -188.45758553278677 muas
dp_q_eta_ref: 9.126363655644703e-06 muas


b_J: 108.15506981721754
defl_ref: 148.29040460362233 muas
dp1_ref: 1.1551595073624242e-05 muas
dp2_ref: 4.3475383830054837e-05 muas
dp_eta_ref: -142.162795523685 muas
dp_q_eta_ref: 9.230532800578858e-06 muas


b_J: 20.941452965187587
defl_ref: 765.8734069919051 muas
dp1_ref: -0.004288705324912875 muas
dp2_ref: 0.0042321746754194756 muas
dp_eta_ref: -734.2262289070495 muas
dp_q_eta_ref: 0.0018016139134735501 muas


b_J: 33.30194388495801
defl_ref: 481.60825997631287 muas
dp1_ref: -0.00119667784825685 muas
dp2_ref: 0.0009433524350547551

b_J: 60.421777447023274
defl_ref: 265.4417028614649 muas
dp1_ref: 1.5530496555613368e-05 muas
dp2_ref: 0.00024540455196162724 muas
dp_eta_ref: -254.47244092971977 muas
dp_q_eta_ref: -3.3708360640120926e-05 muas


b_J: 25.71081872077649
defl_ref: 623.803553406374 muas
dp1_ref: -0.0019987958930849687 muas
dp2_ref: 0.0025133146763740735 muas
dp_eta_ref: -598.0251452003391 muas
dp_q_eta_ref: 0.0007493771864962448 muas


---------- epoch: 2017.1452393686895 ----------


b_J: 4.4245565472298365

defl_tar: 3624.883584449151 muas
dp1_tar: -0.6129350661951707 muas
dp2_tar: -0.16201917497925833 muas
dp_eta_tar: 1912.206185155741 muas
dp_q_eta_tar: -0.4609787729167501 muas


b_J: 81.58860576694823
defl_ref: 196.5769388559871 muas
dp1_ref: -3.481758714228515e-05 muas
dp2_ref: 9.234555522044461e-05 muas
dp_eta_ref: -188.45322268233892 muas
dp_q_eta_ref: 7.105809660100864e-06 muas


b_J: 108.15405940427665
defl_ref: 148.2917900003124 muas
dp1_ref: 1.215385230216431e-05 muas
dp2_ref: 4.25887635539183


---------- epoch: 2017.1452398305944 ----------


b_J: 4.423325358560572

defl_tar: 3625.892534793668 muas
dp1_tar: -0.6095505230453636 muas
dp2_tar: -0.15371839073608617 muas
dp_eta_tar: 1912.7266211890949 muas
dp_q_eta_tar: -0.4521398183520129 muas


b_J: 81.58971414972248
defl_ref: 196.57426834871168 muas
dp1_ref: -3.342772385371868e-05 muas
dp2_ref: 9.187413090017152e-05 muas
dp_eta_ref: -188.4497406814899 muas
dp_q_eta_ref: 5.90589846006597e-06 muas


b_J: 108.15349919778842
defl_ref: 148.29255812072387 muas
dp1_ref: 1.2509730468714935e-05 muas
dp2_ref: 4.205629142721234e-05 muas
dp_eta_ref: -142.16354133019635 muas
dp_q_eta_ref: 1.0260958025831917e-05 muas


b_J: 20.94499142712546
defl_ref: 765.7440195942123 muas
dp1_ref: -0.004055215257563519 muas
dp2_ref: 0.004249439179998737 muas
dp_eta_ref: -734.0953784701028 muas
dp_q_eta_ref: 0.0015865651786238528 muas


b_J: 33.29823813057825
defl_ref: 481.66185817443295 muas
dp1_ref: -0.001139581951203352 muas
dp2_ref: 0.0009552985476428

In [19]:
#shell

In [20]:
#shell_q

Create data

In [21]:
# angular separation
data_y = []
variance =[]
for epoch in epochs:
    
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    r_p = np.zeros((len(target_df), len(reference_df)))
    r_m = np.zeros((len(target_df), len(reference_df)))
    
    var = np.zeros((len(target_df), len(reference_df)))
    err_r_p = np.zeros((len(target_df), len(reference_df)))
    err_r_m = np.zeros((len(target_df), len(reference_df)))
    
    eta_targ = target_df['etaS[deg]'].to_numpy()
    eta_ref = reference_df['etaS[deg]'].to_numpy()
    
    zeta_targ = target_df['zetaS[deg]'].to_numpy()
    zeta_ref = reference_df['zetaS[deg]'].to_numpy()
    
    err_eta_targ = target_df['sig_eta[mas]'].to_numpy() * 1e3
    err_eta_ref = reference_df['sig_eta[mas]'].to_numpy() * 1e3
    
    err_zeta_targ = target_df['sig_zeta[mas]'].to_numpy() * 1e3
    err_zeta_ref = reference_df['sig_zeta[mas]'].to_numpy() * 1e3
    
    x_targ = target_df['starGCdirX'].to_numpy()
    y_targ = target_df['starGCdirY'].to_numpy()
    z_targ = target_df['starGCdirZ'].to_numpy()
    
    x_ref = reference_df['starGCdirX'].to_numpy()
    y_ref = reference_df['starGCdirY'].to_numpy()
    z_ref = reference_df['starGCdirZ'].to_numpy()
    
    alpha_targ = target_df['starRA[deg]_new'].to_numpy() + target_df['dRA[deg]'].to_numpy()
    delta_targ = target_df['starDec[deg]_new'].to_numpy() + target_df['dDEC[deg]'].to_numpy()
    alpha_ref =  reference_df['starRA[deg]_new'].to_numpy() +  reference_df['dRA[deg]'].to_numpy()
    delta_ref = reference_df['starDec[deg]_new'].to_numpy() +  reference_df['dDEC[deg]'].to_numpy()
    
    jup_epoch = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T
    jup_epoch = jup_epoch[0]
    jup_epoch = jup_epoch/LA.norm(jup_epoch)
    
    alpha_jup = stars_epoch['long_jupGC[deg]'].to_numpy()[0]
    delta_jup = stars_epoch['lat_jupGC[deg]'].to_numpy()[0]
    
    eta_J_tar = target_df['etaJup[deg]'].to_numpy()
    zeta_J_tar = target_df['zetaJup[deg]'].to_numpy()
    
    eta_J_ref = reference_df['etaJup[deg]'].to_numpy()
    zeta_J_ref = reference_df['zetaJup[deg]'].to_numpy()
    
    # loop over target
    for i in range(len(target_df)):
        
        r_targ = np.array([x_targ[i], y_targ[i], z_targ[i]]) 
        
        chi_targ = np.arccos(np.dot(r_targ, jup_epoch))
        chi_targ_eta = chi_targ*np.cos(theta_target[i])
        #print(f'chi_targ:       {chi_targ}')
        #print(f'chi_targ_eta:   {chi_targ_eta}')
        
        eta_rel_targ = eta_targ[i] - eta_J_tar[i]
        zeta_rel_targ = zeta_targ[i] - zeta_J_tar[i]
        
        alpha_rel_targ = alpha_targ[i] - alpha_jup
        delta_rel_targ = delta_targ[i] - delta_jup
        
        
        # a_targ_new = np.deg2rad(np.sqrt(eta_rel_targ**2 + zeta_rel_targ**2))
        a_targ_new = np.arccos(np.dot(cartesian(eta_targ[i], zeta_targ[i]), cartesian(eta_J_tar[i], zeta_J_tar[i])))
        #a_targ = np.deg2rad(np.sqrt(alpha_rel_targ**2 + delta_rel_targ**2))
        a_targ = chi_targ
        print(np.rad2deg(a_targ_new)*3600e6 - np.rad2deg(a_targ)*3600e6)
        
        
        # loop over reference
        for j in range(len(reference_df)):
            
            r_ref = np.array([x_ref[j], y_ref[j], z_ref[j]])
            
            theta_rel = np.arctan2((zeta_ref[j] - zeta_targ[i]) ,  (eta_ref[j] - eta_targ[i]))

            
            chi_ref = np.arccos(np.dot(r_ref, jup_epoch))
            chi_ref_eta = chi_ref*np.cos(theta_reference[j])
            
            #print(f'chi_ref:      {chi_ref}')
            #print(f'chi_ref_eta:  {chi_ref_eta}')
            

            #r_p[i, j] = np.sqrt((eta_targ[i] - eta_ref[j])**2 + (zeta_targ[i] - zeta_ref[j])**2)*3600*1e6
            r_p[i, j] = np.abs((eta_targ[i] - eta_ref[j])) *3600*1e6
            
            
            #r_p[i, j] = rad2muas(np.arccos(np.dot(cartesian(eta_targ[i], zeta_targ[i]), cartesian(eta_ref[j], zeta_ref[j]))))
            r_m[i, j] = rad2muas(np.arccos(np.dot(r_targ, r_ref))) * np.cos(theta_rel)
            
            # r_m [i, j] = np.sqrt((alpha_targ[i] - alpha_ref[j])**2 + (delta_targ[i] - delta_ref[j])**2)*3600*1e6
            #r_m[i, j] = chi_ref_eta + chi_targ_eta
            
            
            eta_rel_ref = eta_ref[j] - eta_J_ref[j]
            zeta_rel_ref = zeta_ref[j] - zeta_J_ref[j]
        
            alpha_rel_ref = alpha_ref[j] - alpha_jup
            delta_rel_ref = delta_ref[j] - delta_jup
        
            #a_ref_new = np.deg2rad(np.sqrt(eta_rel_ref**2 + zeta_rel_ref**2))
            a_ref_new = np.arccos(np.dot(cartesian(eta_ref[j], zeta_ref[j]), cartesian(eta_J_ref[j], zeta_J_ref[j])))
            #a_ref = np.deg2rad(np.sqrt(alpha_rel_ref**2 + delta_rel_ref**2))
            a_ref = chi_ref
            print(np.rad2deg(a_ref_new)*3600e6 - np.rad2deg(a_ref)*3600e6)
            
            
            #r_p[i, j] = np.rad2deg((np.cos(a_targ_new) - np.cos(a_targ))/np.sin(a_targ))*3600e6
            #r_m[i, j] = np.rad2deg((np.cos(a_ref_new) - np.cos(a_ref))/np.sin(a_ref))*3600e6
            
            
            err_r_p[i, j] = LA.norm([err_eta_targ[i], err_eta_ref[j]])
            
            var[i, j] = 2*err_eta_targ[i]**2 + err_eta_ref[j]**2
            
            print(f'sep_j: {r_p[i, j]} muas')
            print(f'sep:   {r_m[i, j]} muas\n')
            # print(f'err: {err_r_p[i, j]} muas\n')
            
                
    data_y.append(np.abs(np.abs(r_p) - np.abs(r_m) ))
    variance.append(var)



---------- epoch: 2017.1452385988 ----------

-369.8095567971468
-87804.44500303268
sep_j: 1611016143.6385314 muas
sep:   1611097656.2343552 muas

-88214.1152100563
sep_j: 2113355029.324116 muas
sep:   -2113442157.0222068 muas

-36131.45287781954
sep_j: 416569487.08243227 muas
sep:   416600535.89756024 muas

-44198.2729203701
sep_j: 524275270.5450827 muas
sep:   -524314213.55786115 muas

-69898.57918083668
sep_j: 954604694.2445917 muas
sep:   -954673502.4508289 muas

-95349.23137712479
sep_j: 2055975646.9298055 muas
sep:   -2056060365.4564662 muas

-48555.793892383575
sep_j: 182509072.6526538 muas
sep:   -182521749.18140435 muas

-80834.79198670387
sep_j: 1017207264.1790806 muas
sep:   -1017276479.6565682 muas

-28478.346463620663
sep_j: 240633430.1420571 muas
sep:   240652284.5650612 muas

-62418.28912162781
sep_j: 723843445.1239157 muas
sep:   723891420.2225286 muas

-58331.58797669411
sep_j: 824254353.9789017 muas
sep:   824311752.9333575 muas

-28357.29459810257
sep_j: 319367064.9

In [22]:
data_y

[array([[81512.5958237648  , 87127.6980907917  , 31048.815127968788,
         38943.01277846098 , 68808.20623719692 , 84718.52666068077 ,
         12676.528750538826, 69215.47748756409 , 18854.42300412059 ,
         47975.09861290455 , 57398.9544557333  , 25830.24848419428 ,
         23259.60346221924 , 78236.06668639183 , 35639.03548139334 ]]),
 array([[84330.26587843895 , 85850.34023046494 , 29684.124546408653,
         38130.82553142309 , 68236.39143943787 , 80926.3632452488  ,
         12355.225816845894, 67187.33988821507 , 17863.90493246913 ,
         43572.50194752216 , 52366.67523443699 , 26158.621099591255,
         20472.099108338356, 78497.90892457962 , 36003.032407045364]]),
 array([[83960.56383156776 , 86117.09639596939 , 23860.323750436306,
         39166.80627065897 , 68494.70658636093 , 80554.67613172531 ,
         12165.670934289694, 66842.16782951355 , 18385.250529885292,
         39914.37008714676 , 51817.79338026047 , 27242.629655838013,
         20250.71101385355 ,

## Non linear fit

In [23]:
from scipy.optimize import curve_fit

In [24]:
def model(x_data, gamma, epsilon):
    
    dpsi_mono = x_data[0]
    dpsi_quad = x_data[1]
    
    dr = (1+gamma)*(dpsi_mono + epsilon*dpsi_quad)
    
    return dr

In [25]:
shell = [ep.squeeze() for ep in shell]
shell = np.concatenate(shell)

shell_q = [ep.squeeze() for ep in shell_q]
shell_q = np.concatenate(shell_q)

data_y = [ep.squeeze() for ep in data_y]
data_y = np.concatenate(data_y)

In [26]:
xdata = np.array([shell/2, shell_q/2]).squeeze()
ydata = np.array(data_y).squeeze()

## Monte Carlo

### Define the ranges of the epochs

In [27]:
# entries for each epoch
aa = stars_data['tAFx'].value_counts()
# remove target
aa -= 1
aa

2017.145239    15
2017.145239    15
2017.145239    15
2017.145239    15
2017.145239    15
2017.145240    15
2017.145240    15
2017.145239    14
2017.145240    14
Name: tAFx, dtype: int64

In [28]:
# take cumulative sum
bb = np.cumsum(aa)
bb = bb.to_list()
# add (0,0) element
bb.insert(0, 0)
bb

[0, 15, 30, 45, 60, 75, 90, 105, 119, 133]

In [29]:
# define ranges
epoch_ranges = [(bb[i], bb[i+1]) for i in range(len(bb)-1)]
epoch_ranges

[(0, 15),
 (15, 30),
 (30, 45),
 (45, 60),
 (60, 75),
 (75, 90),
 (90, 105),
 (105, 119),
 (119, 133)]

### Real errors

In [30]:
target_data, reference_data = ref_targ_split(stars_data)

In [31]:
target_errors = target_data['sig_eta[mas]']*1e3
target_errors = target_errors.to_numpy()
target_errors

array([100.9750102539684 ,  85.289853588708  ,  74.0978255734469 ,
        68.72095887140429,  63.4565936178091 ,  66.07312703104961,
        68.4639989220111 ,  66.1654717984665 ,  66.3743315127661 ])

In [32]:
reference_errors = reference_data['sig_eta[mas]']*1e3
reference_errors = reference_errors.to_numpy()

### Run 10000 regressions

In [33]:
# array for gamma and epsilon
gamma = []
epsilon = []

# loop 
for i in range(10000):
    
    rng = np.random.default_rng()
    
    # correlated noise
    y_noise = []
    for i, k in enumerate(epoch_ranges):
        
        # target_noise = rng.normal(scale=14.14)  
        target_noise = rng.normal(scale=target_errors[i])
        
        size = k[1] - k[0]
        
        #noise = target_noise + rng.normal(scale=14.14, size=size)
        #y_noise.append(noise)
        
        # real errors
        for j in range(k[0], k[1]):
            
            noise = target_noise + rng.normal(scale=reference_errors[j])
            
            y_noise.append(noise)
        
    
    # concatenate all arrays
    #y_noise = np.concatenate(y_noise)
    y_noise = np.array(y_noise)
    
    ydata = np.array(shell) + np.array(shell_q) + y_noise
    ydata = ydata.squeeze()
    
    # constrain optimization
    popt, pcov = curve_fit(model, xdata, ydata, p0=(1,1), bounds=(0, [10, 10]))
    
    gamma.append(popt[0])
    epsilon.append(popt[1])
    
# gamma statistic
mean_gamma = np.mean(gamma)
var_gamma = np.var(gamma)

# epsilon statistic
mean_epsilon = np.mean(epsilon)
var_epsilon = np.var(epsilon)

In [34]:
mean_gamma

1.001704947781394

In [35]:
np.sqrt(var_gamma)

0.020745990806155058

In [36]:
mean_epsilon

4.861861348852891

In [37]:
np.sqrt(var_epsilon)

4.961477325494092

## Covariance

In [44]:
rng = np.random.default_rng()

# correlated noise
y_noise = []
for i, k in enumerate(epoch_ranges):
        
    # target_noise = rng.normal(scale=14.14)  
    target_noise = rng.normal(scale=target_errors[i])
        
    size = k[1] - k[0]
        
    #noise = target_noise + rng.normal(scale=14.14, size=size)
    #y_noise.append(noise)
        
    # real errors
    for j in range(k[0], k[1]):
            
        noise = target_noise + rng.normal(scale=reference_errors[j])
            
        y_noise.append(noise)
        
# concatenate all arrays
# y_noise = np.concatenate(y_noise)
y_noise = np.array(y_noise)

ydata = np.array(shell) + np.array(shell_q) +y_noise
ydata = ydata.squeeze()

In [45]:
# covariance matrix
sigma = np.zeros((len(ydata), len(ydata)))

for i in range(len(ydata)):
    for j in range(len(ydata)):
        
        for k in epoch_ranges:
            
            ran = range(k[0], k[1])
            
            if (i in ran) and (j in ran):
                cov = 200
                break
            else:
                cov = 0
            
        sigma[i, j] = 400 if i==j else cov

In [46]:
sigma

array([[400., 200., 200., ...,   0.,   0.,   0.],
       [200., 400., 200., ...,   0.,   0.,   0.],
       [200., 200., 400., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 400., 200., 200.],
       [  0.,   0.,   0., ..., 200., 400., 200.],
       [  0.,   0.,   0., ..., 200., 200., 400.]])

In [47]:
# constrain optimization
popt, pcov = curve_fit(model, xdata, ydata, sigma=sigma, p0=(1,1), bounds=(0, [10, 10]))

In [48]:
popt

array([9.9072936975954717e-01, 2.8999180125960627e-08])

In [49]:
perr = np.sqrt(np.diag(pcov))
perr

array([5.0927823263064879e-02, 1.4011078295472456e+02])

## Test different configurations

In [44]:
# loop over epochs
shell_test = []
shell_q_test = []

for epoch in epochs:
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    bj_target = target_df['b_J'].to_numpy()
    bj_reference = reference_df['b_J'].to_numpy()
    
    
    shell_epoch = np.zeros((len(target_df), len(reference_df)))
    shell_q_epoch = np.zeros((len(target_df), len(reference_df)))
    
    
    # create body
    x_b = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T / 1000# in km
    
    # spin = cartesian(stars_epoch['etaSpin[deg]'].iloc[0], stars_epoch['zetaSpin[deg]'].iloc[0])
    
    body = Body(mass=jupiter.mass,
                pos=x_b[0],
                radius=jupiter.radius,
                J2=jupiter.J2,
                s=np.array([0, 0, 1])) 
    
    ##################
    #
    # print
    #
    ##################
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # loop over target
    x_targ = np.array([target_df['starGCdirX'], target_df['starGCdirY'], target_df['starGCdirZ']]).T
    for i in range(len(target_df)):
        
        # direction
        l0 = x_targ[i]
        
        k_targs = np.array([0.25])
        
        for k_targ in k_targs:
            
            # impact par
            b = (body.pos - l0 * np.dot(body.pos, l0)) * k_targ
            chi = np.arccos(np.dot(body.pos, l0)/np.linalg.norm(body.pos)) * k_targ

            # deflection
            defl_tar = CM_formula(l0, b, chi, body.mass, eps)
            dp_eta_tar = defl_tar * np.cos(theta_target[i])

            # deflection quadrupole
            dp1_tar, dp2_tar = CM_formula(l0, b, chi, body.mass, eps, body.J2, body.radius, body.s)
            dp_q_eta_tar = dp1_tar * np.cos(theta_target[i]) + dp2_tar * np.sin(theta_target[i])


            ##################
            #
            # print
            #
            ##################
            print(f'k_targ: {k_targ}')
            print(f'\nb_J: {bj_target[i]*k_targ}')
            print(f'\ndefl_tar: {np.rad2deg(defl_tar)*3600*1e6} muas')
            print(f'dp1_tar: {np.rad2deg(dp1_tar)*3600*1e6} muas')
            print(f'dp2_tar: {np.rad2deg(dp2_tar)*3600*1e6} muas')
            print(f'dp_eta_tar: {np.rad2deg(dp_eta_tar)*3600*1e6} muas')
            print(f'dp_q_eta_tar: {np.rad2deg(dp_q_eta_tar)*3600*1e6} muas\n')
            
            k_refs = np.array([1])
                  
            for k_ref in k_refs:

                # loop over reference
                x_ref = np.array([reference_df['starGCdirX'], reference_df['starGCdirY'], reference_df['starGCdirZ']]).T
                for j in range(len(reference_df)):

                    # direction
                    l0 = x_ref[j]
                  
                    # impact par
                    b = (body.pos - l0 * np.dot(body.pos, l0)) * k_ref
                    chi = np.arccos(np.dot(body.pos, l0)/np.linalg.norm(body.pos)) * k_ref

                    # deflection
                    defl_ref = CM_formula(l0, b, chi, body.mass, eps)
                    dp_eta_ref = defl_ref * np.cos(theta_reference[i])


                    # deflection quadrupole
                    dp1_ref, dp2_ref = CM_formula(l0, b, chi, body.mass, eps, body.J2, body.radius, body.s)
                    dp_q_eta_ref = dp1_ref * np.cos(theta_reference[j]) + dp2_ref * np.sin(theta_reference[j])
                    
                    diff = dp_eta_tar - dp_eta_ref
                    diff_q = dp_q_eta_tar - dp_q_eta_ref
                    
                    shell_epoch[i, j] = rad2muas(diff)
                    shell_q_epoch[i, j] = rad2muas(diff_q)


                    ##################
                    #
                    # print
                    #
                    ##################
                    print(f'k_ref: {k_ref}')
                    print(f'\nb_J: {bj_reference[j]*k_ref}')
                    print(f'defl_ref: {np.rad2deg(defl_ref)*3600*1e6} muas')
                    print(f'dp1_ref: {np.rad2deg(dp1_ref)*3600*1e6} muas')
                    print(f'dp2_ref: {np.rad2deg(dp2_ref)*3600*1e6} muas')
                    print(f'dp_eta_ref: {np.rad2deg(dp_eta_ref)*3600*1e6} muas')
                    print(f'dp_q_eta_ref: {np.rad2deg(dp_q_eta_ref)*3600*1e6} muas\n')
                    
                
                        
                shell_test.append(shell_epoch)
                shell_q_test.append(shell_q_epoch)


---------- epoch: 2017.1452385988 ----------

k_targ: 0.25

b_J: 1.1069109626388967

defl_tar: 15272.507817043726 muas
dp1_tar: 79.89069518600942 muas
dp2_tar: -168.50485906355055 muas
dp_eta_tar: 8054.941577363626 muas
dp_q_eta_tar: -101.02759023628188 muas

k_ref: 1

b_J: 81.58707469920164
defl_ref: 207.1945725642207 muas
dp1_ref: -0.0003935378213349119 muas
dp2_ref: -0.00024710256124508954 muas
dp_eta_ref: -198.63294413282023 muas
dp_q_eta_ref: 0.0004475749590271111 muas

k_ref: 1

b_J: 108.15506981721754
defl_ref: 156.29905966767274 muas
dp1_ref: -0.0001989023180400409 muas
dp2_ref: 2.4547356941759885e-05 muas
dp_eta_ref: -149.84051948251835 muas
dp_q_eta_ref: -0.00019992397350955416 muas

k_ref: 1

b_J: 20.941452965187587
defl_ref: 807.1988827712722 muas
dp1_ref: -0.015778194978545008 muas
dp2_ref: -0.02254580960028409 muas
dp_eta_ref: -773.8440664795128 muas
dp_q_eta_ref: 0.024512027028917235 muas

k_ref: 1

b_J: 33.30194388495801
defl_ref: 507.60528378643096 muas
dp1_ref: -0.00

dp_eta_ref: -198.6244117443117 muas
dp_q_eta_ref: 0.00044752374540609423 muas

k_ref: 1

b_J: 108.15349919778842
defl_ref: 156.30155018512096 muas
dp1_ref: -0.00019891001032241795 muas
dp2_ref: 2.4563236468277297e-05 muas
dp_eta_ref: -149.84151714226115 muas
dp_q_eta_ref: -0.00019993301950296927 muas

k_ref: 1

b_J: 20.94499142712546
defl_ref: 807.0676865008908 muas
dp1_ref: -0.015764900832358538 muas
dp2_ref: -0.02253873750588137 muas
dp_eta_ref: -773.7111144358975 muas
dp_q_eta_ref: 0.02449863732852956 muas

k_ref: 1

b_J: 33.29823813057825
defl_ref: 507.6601525496354 muas
dp1_ref: -0.003325122192267768 muas
dp2_ref: -0.00599293875318626 muas
dp_eta_ref: -486.6782662143452 muas
dp_q_eta_ref: -0.005935771592160662 muas

k_ref: 1

b_J: 50.71046426681159
defl_ref: 333.3467500104861 muas
dp1_ref: -0.0018531205872957947 muas
dp2_ref: -0.0005788427678407462 muas
dp_eta_ref: -319.569336944223 muas
dp_q_eta_ref: -0.0019216416231756248 muas

k_ref: 1

b_J: 31.855123859957096
defl_ref: 530.659

In [45]:
shell_test

[array([[8253.574521496446, 8204.782096846144, 8828.785643843139,
         8541.571756562082, 8374.498344978652, 8201.67932909001 ,
         8563.720206902863, 8331.546245872447, 8870.166356718177,
         8434.683933170161, 8465.549633080576, 8822.637509425836,
         8985.087172564665, 8323.155947597108, 8685.288277373322]]),
 array([[8254.575924601324, 8205.784628635856, 8829.772231739971,
         8542.58059674523 , 8375.502537240505, 8202.681670514488,
         8564.716575803368, 8332.548535083586, 8871.18174250225 ,
         8435.68201003716 , 8466.548767671653, 8823.65588093487 ,
         8986.09299542111 , 8324.15707342523 , 8686.280583780488]]),
 array([[8255.613647019289, 8206.82353812183 , 8830.794449784662,
         8543.625423773392, 8376.542908874184, 8203.720393123209,
         8565.748894763889, 8333.587052571449, 8872.232733886334,
         8436.716190305515, 8467.583970157873, 8824.709916999784,
         8987.13427957537 , 8325.19443642954 , 8687.308693848152]]),
 

In [46]:
shell_test = [ep.squeeze() for ep in shell_test]
shell_test = np.concatenate(shell_test)

shell_q_test = [ep.squeeze() for ep in shell_q_test]
shell_q_test = np.concatenate(shell_q_test)

xdata = np.array([shell_test/2, shell_q_test/2]).squeeze()

rng = np.random.default_rng()

# correlated noise
y_noise = []
for i, k in enumerate(epoch_ranges):
        
    # target_noise = rng.normal(scale=14.14)  
    target_noise = rng.normal(scale=target_errors[i])
        
    size = k[1] - k[0]
        
    #noise = target_noise + rng.normal(scale=14.14, size=size)
    #y_noise.append(noise)
        
    # real errors
    for j in range(k[0], k[1]):
            
        noise = target_noise + rng.normal(scale=reference_errors[j])
            
        y_noise.append(noise)
        
# concatenate all arrays
# y_noise = np.concatenate(y_noise)
y_noise = np.array(y_noise)

ydata = np.array(shell_test) + np.array(shell_q_test) +y_noise
ydata = ydata.squeeze()

# covariance matrix
sigma = np.zeros((len(ydata), len(ydata)))

for i in range(len(ydata)):
    for j in range(len(ydata)):
        
        for k in epoch_ranges:
            
            ran = range(k[0], k[1])
            
            if (i in ran) and (j in ran):
                cov = 200
                break
            else:
                cov = 0
            
        sigma[i, j] = 400 if i==j else cov
        
# constrain optimization
popt, pcov = curve_fit(model, xdata, ydata, sigma=sigma, p0=(1,1), bounds=(0, [10, 10]))

In [47]:
popt

array([0.999066259294269 , 0.8972119918718842])

In [48]:
perr = np.sqrt(np.diag(pcov))
perr

array([0.04308712088888742, 1.8111489724839476 ])

## Data last transit

In [49]:
# path
path = 'Stars_GareqEvent2017_lastTransit.dat'

# read, use delimiter='\s+' to choose every white space as delimiter
stars_data_last = pd.read_table(path, delimiter='\s+')

# display
stars_data_last.head(10)

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg]
0,2017.150921,2017.15092,3631060567169213952,0.545830,0.078291,-0.244815,0.270846,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.072637,-7.882125
1,2017.150921,2017.15092,3631150658403448192,0.286463,0.093628,-0.448373,0.200103,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,200.912319,-7.593197
2,2017.150921,2017.15092,3631485528413582336,-0.588215,0.084727,-0.037523,0.263414,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.463499,-6.796371
3,2017.150921,2017.15092,3631167116717952256,-0.052080,0.069137,-0.393695,0.173877,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.022035,-7.267964
4,2017.150921,2017.15092,3631453505137203968,-0.365379,0.100718,-0.266164,0.294687,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.199973,-6.979410
5,2017.150921,2017.15092,3631356709458613120,-0.182034,0.075145,-0.419366,0.194678,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.017753,-7.135571
6,2017.150921,2017.15092,3631191099815174272,0.102827,0.076954,0.092277,0.241782,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.480483,-7.499324
7,2017.150921,2017.15092,3631256894418771584,-0.132568,0.084850,-0.095727,0.255418,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.331641,-7.236695
8,2017.150921,2017.15092,3631356404516663936,-0.145366,0.100314,-0.459708,0.225759,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,200.971634,-7.165225
9,2017.150921,2017.15092,3631266072764288000,-0.258396,0.077476,-0.064199,0.237267,-0.022152,-0.08978,-86.390812,14.051664,-6.521604e+11,-2.545235e+11,-9.026000e+10,-1.362128e+11,5.600714e+10,2.435285e+10,201.319579,-7.346634,157.648824,9.389083,201.383427,-7.117590


In [50]:
# star and jupiter coordinates
star = np.array([stars_data_last['etaS[deg]'], stars_data_last['zetaS[deg]']])
jup = np.array([stars_data_last['etaJup[deg]'], stars_data_last['zetaJup[deg]']])

# impact parameter in units of jupiter radius
stars_data_last['b_J'] = np.linalg.norm(star - jup, axis=0) * 3600 / r_jup_arc

In [51]:
stars_data_last['tAFx'].unique()

array([2017.150920494697 , 2017.1509206485832, 2017.150920802584 ,
       2017.1509209565909, 2017.1509211106013, 2017.1509212646056,
       2017.150921418616 , 2017.150921572619 , 2017.1509217265343])

### Verify starId overlap

In [52]:
len(stars_data_last['starId'].unique())

27

In [53]:
len(stars_data['starId'].unique())

16

In [54]:
for Sid in stars_data['starId'].unique():
    
    if Sid in stars_data_last['starId'].unique():
        print(f'{Sid} Yes')
    else:
        print(f'{Sid} NO')

3631075715518049024 NO
3631485528413582336 Yes
3631191099815174272 Yes
3631256894418771584 Yes
3631266072764288000 Yes
3631298199119733504 Yes
3631241265033227392 NO
3631239925003381248 Yes
3631255107712373376 Yes
3631145740665759616 NO
3631186628753669888 NO
3631095407943986048 Yes
3631244559272698112 Yes
3631244937229820800 Yes
3631089188831339264 Yes
3631096438736162944 Yes


### Prepare new sets

In [55]:
# need o choose only the stars in common

# save common id
common_id = []
for Sid in stars_data['starId'].unique():
    
    if Sid in stars_data_last['starId'].unique():
        
        common_id.append(Sid)

# take common id from first set        
filt = [Sid in common_id for Sid in stars_data['starId']]
star_new = stars_data.loc[filt].copy()

# take common id from second set        
filt = [Sid in common_id for Sid in stars_data_last['starId']]
star_new_last = stars_data_last.loc[filt].copy()

In [56]:
# verify that the target star is in the set and save the value
target, _ = ref_targ_split(star_new)
target_id = target['starId'].unique()[0]

In [57]:
# define the target star in the second set
star_new_last['thr'] = (star_new_last['starId'] == target_id).values

In [58]:
star_new['tAFx'].value_counts()

2017.145239    12
2017.145239    12
2017.145239    12
2017.145239    12
2017.145239    12
2017.145240    12
2017.145240    12
2017.145239    11
2017.145240    11
Name: tAFx, dtype: int64

In [59]:
star_new_last['tAFx'].value_counts()

2017.150920    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150922    12
2017.150922    12
Name: tAFx, dtype: int64

In [60]:
# remove the extra reference

print(star_new['tAFx'].unique())
print(star_new_last['tAFx'].unique())

[2017.1452385988    2017.145238752673  2017.1452389066772
 2017.1452390606796 2017.145239214684  2017.1452393686895
 2017.145239522684  2017.1452396766856 2017.1452398305944]
[2017.150920494697  2017.1509206485832 2017.150920802584
 2017.1509209565909 2017.1509211106013 2017.1509212646056
 2017.150921418616  2017.150921572619  2017.1509217265343]


In [61]:
# select the missing star
filt = star_new['tAFx'] == 2017.1452396766856
star_epoch = star_new.loc[filt]

filt = star_new_last['tAFx'] == 2017.150921572619
star_epoch_last = star_new_last.loc[filt]

missing_id = 0
for Sid in star_epoch_last['starId']:
    
    if Sid not in star_epoch['starId']:
        missing_id = Sid    

In [62]:
missing_id

3631096438736162944

In [63]:
# select index to drop
indices = []

filt = star_new_last['starId']==missing_id
df_temp = star_new_last.loc[filt]

filt = df_temp['tAFx']==2017.150921572619
indices.append(df_temp.loc[filt].index[0])

filt = star_new_last['starId']==missing_id
df_temp = star_new_last.loc[filt]

filt = df_temp['tAFx']==2017.1509217265343
indices.append(df_temp.loc[filt].index[0])

In [64]:
star_new_last.drop(indices, inplace=True)

In [65]:
star_new_last['tAFx'].value_counts()

2017.150920    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150921    12
2017.150922    11
2017.150922    11
Name: tAFx, dtype: int64

In [66]:
# verify minimum impact parameter
star_new_last['b_J'].min()

12.86365838612743

In [67]:
# verify target impact parameter
target, _ = ref_targ_split(star_new_last)
target['b_J']

14     16.645241
41     16.645739
68     16.646468
95     16.647091
122    16.647512
149    16.648229
176    16.648753
203    16.649399
229    16.650077
Name: b_J, dtype: float64

In [68]:
# drop stars with impact parameter less than 16 in the last transit

filt = star_new_last['b_J']<=16.5
df_temp = star_new_last.loc[filt]

# select index to drop
id_drop = df_temp['starId'].values

filt = [Sid not in id_drop for Sid in star_new['starId']]
star_new = star_new.loc[filt].copy()

filt = [Sid not in id_drop for Sid in star_new_last['starId']]
star_new_last = star_new_last.loc[filt].copy()

# verify
star_new_last['b_J'].min()

16.64524069623232

In [69]:
# order the two datasets
star_new = star_new.sort_values(by=['tAFx', 'starId'])
star_new_last = star_new_last.sort_values(by=['tAFx', 'starId'])

### Deflections

In [79]:
epochs = star_new['tAFx'].unique()

# differences
shell=[]
shell_q=[]

# loop over epochs
for epoch in epochs:
    
    # take only data for the given epoch
    filt = star_new['tAFx'] == epoch
    stars_epoch = star_new.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    bj_target = target_df['b_J'].to_numpy()
    bj_reference = reference_df['b_J'].to_numpy()
    
    
    shell_epoch = np.zeros((len(target_df), len(reference_df)))
    shell_q_epoch = np.zeros((len(target_df), len(reference_df)))
    
    
    # create body
    #x_b = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T / 1000# in km
    x_b = cartesian(stars_epoch['etaJup[deg]'], stars_epoch['zetaJup[deg]']).T * 5*AU
    
    spin = cartesian(stars_epoch['etaSpin[deg]'].iloc[0], stars_epoch['zetaSpin[deg]'].iloc[0]) 
    # spin = np.array([0, 0, 1])
    
    body = Body(mass=jupiter.mass,
                pos=x_b[0],
                radius=jupiter.radius,
                J2=jupiter.J2,
                s=spin) 
    
    ##################
    #
    # print
    #
    ##################
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # loop over target
    #x_targ = np.array([target_df['starGCdirX'], target_df['starGCdirY'], target_df['starGCdirZ']]).T
    x_targ = cartesian(target_df['etaS[deg]'], target_df['zetaS[deg]']).T
    for i in range(len(target_df)):
        
        # direction
        l0 = x_targ[i]
        
        # deflection
        defl_tar = deflection_mod3(l0, body.pos, eps, body.mass)
        dp_eta_tar = defl_tar * np.cos(theta_target[i])
        
        # deflection quadrupole
        dp1_tar, dp2_tar = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
        dp_q_eta_tar = dp1_tar * np.cos(theta_target[i]) + dp2_tar * np.sin(theta_target[i])
        
        ##################
        #
        # print
        #
        ##################
        print(f'\nb_J: {bj_target[i]}')
        print(f'\ndefl_tar: {np.rad2deg(defl_tar)*3600*1e6} muas')
        print(f'dp1_tar: {np.rad2deg(dp1_tar)*3600*1e6} muas')
        print(f'dp2_tar: {np.rad2deg(dp2_tar)*3600*1e6} muas')
        print(f'dp_eta_tar: {np.rad2deg(dp_eta_tar)*3600*1e6} muas')
        print(f'dp_q_eta_tar: {np.rad2deg(dp_q_eta_tar)*3600*1e6} muas\n')
        # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
        
        
        # loop over reference
        #x_ref = np.array([reference_df['starGCdirX'], reference_df['starGCdirY'], reference_df['starGCdirZ']]).T
        x_ref = cartesian(reference_df['etaS[deg]'], reference_df['zetaS[deg]']).T
        for j in range(len(reference_df)):
            
            # direction
            l0 = x_ref[j]
        
            # deflection
            defl_ref = deflection_mod3(l0, body.pos, eps, body.mass)
            dp_eta_ref = defl_ref * np.cos(theta_reference[i])
            
            diff = dp_eta_tar - dp_eta_ref
            
            # deflection quadrupole
            dp1_ref, dp2_ref = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
            dp_q_eta_ref = dp1_ref * np.cos(theta_reference[j]) + dp2_ref * np.sin(theta_reference[j])
            
            diff_q = dp_q_eta_tar - dp_q_eta_ref
            
            shell_epoch[i, j] = np.rad2deg(diff)*3600*1e6
            shell_q_epoch[i, j] = np.rad2deg(diff_q)*3600*1e6
            
            ##################
            #
            # print
            #
            ##################
            print(f'\nb_J: {bj_reference[j]}')
            print(f'defl_ref: {np.rad2deg(defl_ref)*3600*1e6} muas')
            print(f'dp1_ref: {np.rad2deg(dp1_ref)*3600*1e6} muas')
            print(f'dp2_ref: {np.rad2deg(dp2_ref)*3600*1e6} muas')
            print(f'dp_eta_ref: {np.rad2deg(dp_eta_ref)*3600*1e6} muas')
            print(f'dp_q_eta_ref: {np.rad2deg(dp_q_eta_ref)*3600*1e6} muas\n')
            # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
            
    
    shell.append(shell_epoch)
    shell_q.append(shell_q_epoch)


---------- epoch: 2017.1452385988 ----------


b_J: 4.427643850555587

defl_tar: 3622.356028983728 muas
dp1_tar: -0.6180035333819498 muas
dp2_tar: -0.17597130634532407 muas
dp_eta_tar: 1910.4829760383607 muas
dp_q_eta_tar: -0.47545065748617776 muas


b_J: 60.420753927963304
defl_ref: 265.4461994492952 muas
dp1_ref: 1.1961534395302872e-05 muas
dp2_ref: 0.0002487010534377759 muas
dp_eta_ref: -264.7139765186222 muas
dp_q_eta_ref: -3.038831071839794e-05 muas


b_J: 39.46733202846395
defl_ref: 406.3737318471006 muas
dp1_ref: -0.00013897293769160872 muas
dp2_ref: 0.0008860563466263205 muas
dp_eta_ref: -405.25276584533106 muas
dp_q_eta_ref: -1.5714286211113968e-05 muas


b_J: 25.708927751464522
defl_ref: 623.8494361255035 muas
dp1_ref: -0.0020620516884568402 muas
dp2_ref: 0.002514993871053158 muas
dp_eta_ref: -622.1285719226398 muas
dp_q_eta_ref: 0.0008062299315568148 muas


b_J: 20.941452965187587
defl_ref: 765.8734069919051 muas
dp1_ref: -0.004288705324912875 muas
dp2_ref: 0.00423217467541

In [80]:
# useful values
eta_s = np.deg2rad(star_new_last['etaS[deg]'])
zeta_s = np.deg2rad(star_new_last['zetaS[deg]'])
eta_J = np.deg2rad(star_new_last['etaJup[deg]'])
zeta_J = np.deg2rad(star_new_last['zetaJup[deg]'])

# gnomonic coordinates
# cos_psi = np.sin(zeta_J)*np.sin(zeta_s) + np.cos(zeta_J)*np.cos(zeta_s)*np.cos(eta_J - eta_s)
# the previous factor can be neglected because we take the fraction
x_g = np.cos(zeta_J)*np.sin(eta_J - eta_s)
y_g = np.sin(zeta_J)*np.cos(zeta_s) - np.cos(zeta_J)*np.sin(zeta_s)*np.cos(eta_J - eta_s)
theta = np.arctan2(y_g,x_g)
star_new_last['theta'] = theta

In [81]:
epochs_last = star_new_last['tAFx'].unique()

# differences
shell_last=[]
shell_q_last=[]

# loop over epochs
for epoch in epochs_last:
    
    # take only data for the given epoch
    filt = star_new_last['tAFx'] == epoch
    stars_epoch = star_new_last.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    # need to order by starId the reference_df
    reference_df = reference_df.sort_values(by=['starId'])
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    bj_target = target_df['b_J'].to_numpy()
    bj_reference = reference_df['b_J'].to_numpy()
    
    
    shell_epoch = np.zeros((len(target_df), len(reference_df)))
    shell_q_epoch = np.zeros((len(target_df), len(reference_df)))
    
    
    # create body
    # x_b = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T / 1000# in km
    x_b = cartesian(stars_epoch['etaJup[deg]'], stars_epoch['zetaJup[deg]']).T * 5*AU
    
    spin = cartesian(stars_epoch['etaSpin[deg]'].iloc[0], stars_epoch['zetaSpin[deg]'].iloc[0]) 
    # spin = np.array([0, 0, 1])
    
    body = Body(mass=jupiter.mass,
                pos=x_b[0],
                radius=jupiter.radius,
                J2=jupiter.J2,
                s=spin) 
    
    
    ##################
    #
    # print
    #
    ##################
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # loop over target
    # x_targ = np.array([target_df['starGCdirX'], target_df['starGCdirY'], target_df['starGCdirZ']]).T
    x_targ = cartesian(target_df['etaS[deg]'], target_df['zetaS[deg]']).T
    for i in range(len(target_df)):
        
        # direction
        l0 = x_targ[i]
        
        
        # deflection
        defl_tar = deflection_mod3(l0, body.pos, eps, body.mass)
        dp_eta_tar = defl_tar * np.cos(theta_target[i])
        
        # deflection quadrupole
        dp1_tar, dp2_tar = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
        dp_q_eta_tar = dp1_tar * np.cos(theta_target[i]) + dp2_tar * np.sin(theta_target[i])
        
        
        ##################
        #
        # print
        #
        ##################
        print(f'\nb_J: {bj_target[i]}')
        print(f'\ndefl_tar: {np.rad2deg(defl_tar)*3600*1e6} muas')
        print(f'dp1_tar: {np.rad2deg(dp1_tar)*3600*1e6} muas')
        print(f'dp2_tar: {np.rad2deg(dp2_tar)*3600*1e6} muas')
        print(f'dp_eta_tar: {np.rad2deg(dp_eta_tar)*3600*1e6} muas')
        print(f'dp_q_eta_tar: {np.rad2deg(dp_q_eta_tar)*3600*1e6} muas\n')
        # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
        
        
        # loop over reference
        # x_ref = np.array([reference_df['starGCdirX'], reference_df['starGCdirY'], reference_df['starGCdirZ']]).T
        x_ref = cartesian(reference_df['etaS[deg]'], reference_df['zetaS[deg]']).T
        for j in range(len(reference_df)):
            
            # direction
            l0 = x_ref[j]
            
        
            # deflection
            defl_ref = deflection_mod3(l0, body.pos, eps, body.mass)
            dp_eta_ref = defl_ref * np.cos(theta_reference[i])
            
            diff = dp_eta_tar - dp_eta_ref
            
            # deflection quadrupole
            dp1_ref, dp2_ref = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
            dp_q_eta_ref = dp1_ref * np.cos(theta_reference[j]) + dp2_ref * np.sin(theta_reference[j])
            
            diff_q = dp_q_eta_tar - dp_q_eta_ref
            
            shell_epoch[i, j] = np.rad2deg(diff)*3600*1e6
            shell_q_epoch[i, j] = np.rad2deg(diff_q)*3600*1e6
            
            ##################
            #
            # print
            #
            ##################
            print(f'\nb_J: {bj_reference[j]}')
            print(f'defl_ref: {np.rad2deg(defl_ref)*3600*1e6} muas')
            print(f'dp1_ref: {np.rad2deg(dp1_ref)*3600*1e6} muas')
            print(f'dp2_ref: {np.rad2deg(dp2_ref)*3600*1e6} muas')
            print(f'dp_eta_ref: {np.rad2deg(dp_eta_ref)*3600*1e6} muas')
            print(f'dp_q_eta_ref: {np.rad2deg(dp_q_eta_ref)*3600*1e6} muas\n')
            # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
            
    
    shell_last.append(shell_epoch)
    shell_q_last.append(shell_q_epoch)


---------- epoch: 2017.150920494697 ----------


b_J: 16.64524069623232

defl_tar: 963.5486642401179 muas
dp1_tar: 0.047438292390281946 muas
dp2_tar: 0.00036174503649384165 muas
dp_eta_tar: -230.8187925867043 muas
dp_q_eta_tar: -0.01171509031347362 muas


b_J: 72.92512858439395
defl_ref: 219.93024512801264 muas
dp1_ref: -6.405904703897526e-05 muas
dp2_ref: -0.0005608477681036326 muas
dp_eta_ref: -194.77189303232242 muas
dp_q_eta_ref: 0.00031720811132780714 muas


b_J: 54.21963717657678
defl_ref: 295.8055556584705 muas
dp1_ref: 0.00030497939293256277 muas
dp2_ref: -0.0013388524611790986 muas
dp_eta_ref: -261.9676434750626 muas
dp_q_eta_ref: 0.0005697320231870622 muas


b_J: 43.88728275039294
defl_ref: 365.44712327969137 muas
dp1_ref: 0.001729225774641864 muas
dp2_ref: -0.0019265040225104348 muas
dp_eta_ref: -323.6427439207902 muas
dp_q_eta_ref: 0.0004483627044037387 muas


b_J: 39.74891861723387
defl_ref: 403.49487257253855 muas
dp1_ref: 0.002637789903399954 muas
dp2_ref: -0.0022764187


b_J: 72.9275514907242
defl_ref: 219.92293823144448 muas
dp1_ref: -6.42000135538823e-05 muas
dp2_ref: -0.0005612587411782803 muas
dp_eta_ref: -194.76190752561882 muas
dp_q_eta_ref: 0.0003175398985286499 muas


b_J: 54.222161333831444
defl_ref: 295.79178522464775 muas
dp1_ref: 0.0003049645545018748 muas
dp2_ref: -0.0013399044817956932 muas
dp_eta_ref: -261.9507213937525 muas
dp_q_eta_ref: 0.0005704217007241764 muas


b_J: 43.8906198956427
defl_ref: 365.41933707888825 muas
dp1_ref: 0.0017300461677799976 muas
dp2_ref: -0.0019281386213671951 muas
dp_eta_ref: -323.61229669154886 muas
dp_q_eta_ref: 0.0004492214180549789 muas


b_J: 39.75189285719182
defl_ref: 403.4646829833872 muas
dp1_ref: 0.0026391367727271052 muas
dp2_ref: -0.0022785466597750317 muas
dp_eta_ref: -357.304935578695 muas
dp_q_eta_ref: 0.0003849493779459973 muas


b_J: 25.746357708310388
defl_ref: 622.9422965719949 muas
dp1_ref: 0.0019927710010938455 muas
dp2_ref: 0.012673418153507218 muas
dp_eta_ref: -551.6724673397641 muas


### New simulation

In [82]:
# entries for each epoch
aa = star_new['tAFx'].value_counts()
# remove target
aa -= 1
# take cumulative sum
bb = np.cumsum(aa)
bb = bb.to_list()
# add (0,0) element
bb.insert(0, 0)
# define ranges
epoch_ranges = [(bb[i], bb[i+1]) for i in range(len(bb)-1)]
epoch_ranges

[(0, 10),
 (10, 20),
 (20, 30),
 (30, 40),
 (40, 50),
 (50, 60),
 (60, 70),
 (70, 79),
 (79, 88)]

In [83]:
# errors
target_data, reference_data = ref_targ_split(star_new)
target_data_last, reference_data_last = ref_targ_split(star_new_last)

target_errors = target_data['sig_eta[mas]']*1e3
target_errors = target_errors.to_numpy()

target_errors_last = target_data_last['sig_eta[mas]']*1e3
target_errors_last = target_errors_last.to_numpy()

reference_errors = reference_data['sig_eta[mas]']*1e3
reference_errors = reference_errors.to_numpy()

reference_errors_last = reference_data_last['sig_eta[mas]']*1e3
reference_errors_last = reference_errors_last.to_numpy()

In [84]:
# shell
shell = [ep.squeeze() for ep in shell]
shell = np.concatenate(shell)

shell_q = [ep.squeeze() for ep in shell_q]
shell_q = np.concatenate(shell_q)

# shell_last
shell_last = [ep.squeeze() for ep in shell_last]
shell_last = np.concatenate(shell_last)

shell_q_last = [ep.squeeze() for ep in shell_q_last]
shell_q_last = np.concatenate(shell_q_last)

xdata = np.array([(shell-shell_last)/2, (shell_q-shell_q_last)/2]).squeeze()

In [91]:
# array for gamma and epsilon
gamma = []
epsilon = []

# loop 
for i in range(10000):
    
    rng = np.random.default_rng()
    
    # correlated noise
    y_noise = []
    for i, k in enumerate(epoch_ranges):
        
        # target_noise = rng.normal(scale=14.14)  
        err_targ = np.sqrt(target_errors[i]**2 + target_errors_last[i]**2)
        target_noise = rng.normal(scale=err_targ)
        
        # real errors
        for j in range(k[0], k[1]):
            
            err_ref = np.sqrt(reference_errors[j]**2 + reference_errors_last[j]**2)
            noise = target_noise + rng.normal(scale=reference_errors[j])
            
            y_noise.append(noise)
        
    
    # concatenate all arrays
    #y_noise = np.concatenate(y_noise)
    y_noise = np.array(y_noise)
    
    ydata = np.array(shell-shell_last) + np.array(shell_q-shell_q_last) + y_noise
    ydata = ydata.squeeze()
    
    # constrain optimization
    popt, pcov = curve_fit(model, xdata, ydata, p0=(1,1), bounds=(0, [10, 10]))
    
    gamma.append(popt[0])
    epsilon.append(popt[1])
    
# gamma statistic
mean_gamma = np.mean(gamma)
var_gamma = np.var(gamma)

# epsilon statistic
mean_epsilon = np.mean(epsilon)
var_epsilon = np.var(epsilon)

In [92]:
mean_gamma

1.0016088049175658

In [93]:
np.sqrt(var_gamma)

0.02967162236547107

In [94]:
mean_epsilon

4.983811519173579

In [95]:
np.sqrt(var_epsilon)

4.970328874336358

In [96]:
rng = np.random.default_rng()

# correlated noise
y_noise = []
for i, k in enumerate(epoch_ranges):
        
    # target_noise = rng.normal(scale=14.14)  
    err_targ = np.sqrt(target_errors[i]**2 + target_errors_last[i]**2)
    target_noise = rng.normal(scale=err_targ)
        
    # real errors
    for j in range(k[0], k[1]):
            
        err_ref = np.sqrt(reference_errors[j]**2 + reference_errors_last[j]**2)
        noise = target_noise + rng.normal(scale=reference_errors[j])
            
        y_noise.append(noise)
        
# concatenate all arrays
# y_noise = np.concatenate(y_noise)
y_noise = np.array(y_noise)

ydata = np.array(shell-shell_last) + np.array(shell_q-shell_q_last) +y_noise
ydata = ydata.squeeze()

# covariance matrix
sigma = np.zeros((len(ydata), len(ydata)))

for i in range(len(ydata)):
    for j in range(len(ydata)):
        
        for k in epoch_ranges:
            
            ran = range(k[0], k[1])
            
            if (i in ran) and (j in ran):
                cov = 200
                break
            else:
                cov = 0
            
        sigma[i, j] = 400 if i==j else cov

# constrain optimization
popt, pcov = curve_fit(model, xdata, ydata, sigma=sigma, p0=(1,1), bounds=(0, [10, 10]))

In [97]:
popt

array([1.0094253548517935, 9.999999999858062 ])

In [98]:
np.sqrt(np.diag(pcov))

array([5.7842327932279308e-02, 1.4874070407086822e+02])

### Separations

In [ ]:
# angular separation
data_y = []
variance =[]

# choose 1 epoch
epochs = star_new['tAFx'].unique()
epochs_last = star_new_last['tAFx'].unique()
  
# take only data for the given epoch
filt = star_new['tAFx'] == epochs[8]
stars_epoch = star_new.loc[filt]

filt = star_new_last['tAFx'] == epochs_last[0]
stars_epoch_last = star_new_last.loc[filt]
    
# split into target and reference
target_df, reference_df = ref_targ_split(stars_epoch)
target_df_last, reference_df_last = ref_targ_split(stars_epoch_last)

# need to order by starId the reference_df
reference_df = reference_df.sort_values(by=['starId'])
reference_df_last = reference_df_last.sort_values(by=['starId'])
    
eta_targ_p = rad2muas(target_df['etaS[deg]'].to_numpy())
eta_ref_p = rad2muas(reference_df['etaS[deg]'].to_numpy())

eta_targ_m = rad2muas(target_df_last['etaS[deg]'].to_numpy())
eta_ref_m = rad2muas(reference_df_last['etaS[deg]'].to_numpy())

zeta_targ_p = rad2muas(target_df['zetaS[deg]'].to_numpy())
zeta_ref_p = rad2muas(reference_df['zetaS[deg]'].to_numpy())

zeta_targ_m = rad2muas(target_df_last['zetaS[deg]'].to_numpy())
zeta_ref_m = rad2muas(reference_df_last['zetaS[deg]'].to_numpy())

# loop over target
for i in range(len(target_df)):
    
    # loop over reference
    for j in range(len(reference_df)):
        
        d_1 = np.sqrt((eta_ref_p[j] - eta_targ_p[i])**2 + (zeta_ref_p[j] - zeta_targ_p[i])**2) 
        d_2 = np.sqrt((eta_ref_m[j] - eta_targ_m[i])**2 + (zeta_ref_m[j] - zeta_targ_m[i])**2)
        data_y.append(d_1 - d_2)
        
        print(f'eta_ref_p: {eta_ref_p[j]}')
        print(f'eta_targ_p: {eta_targ_p[i]}')
        print(f'eta_ref_m: {eta_ref_m[j]}')
        print(f'eta_targ_m: {eta_targ_m[i]}')
        print(f'd1: {d_1}')
        print(f'd2: {d_2}')
        print(f'd1-d2: {d_1-d_2}\n')
